In [169]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
import numpy as np
import pandas as pd
from q_value_calc_crosslinks import calcQ
import matplotlib.pyplot as plt
from functions import get_target_id, get_datasets, rerank, get_top_indices, get_last_top_indices, get_nth_top_scan
import re
import os

In [170]:
i = 1
n_iters = 5
pep_filter = False
#decoy = "second" 
decoy = "bottom" 
#decoy = "last" 
#decoy = "bottom+last"
#decoy = "bottom+second" 
class_threshold = 500
dataset = get_datasets()

In [171]:
input_filename = f"{dataset[i]['file']}.pkl"
input_file = f"../data/{dataset[i]['type']}/{dataset[i]['name']}/{input_filename}" 

In [172]:
# read original dataframes
original_df = pd.read_pickle(input_file)
display(original_df)

,SpecId,PSMId,Label,Score,ScanNr,Peptide,peplen,ExpMass,charge2,charge3,...,G_346.055250000000001,U_113.035089999999997,U_307.033110000000022,NuXL:Da difference,NuXL:z1 mass,NuXL:z2 mass,NuXL:z3 mass,NuXL:z4 mass,class-specific_q-val,cum_target_id
23703,controllerType=0 controllerNumber=1 scan=8757,1,1,165.613602,8757,GGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGK,40,1075.0995,0,1,...,0.0,0.0,0.000000,-0.000784,3223.281594,1612.144435,1075.098716,806.575856,0.0,509.0
16189,controllerType=0 controllerNumber=1 scan=6694,1,1,156.726578,6694,EDVFVHQTAIKK,12,681.9542,0,1,...,0.0,0.0,1.523579,-0.000664,2043.846057,1022.426666,681.953536,511.716971,0.0,723.0
26723,controllerType=0 controllerNumber=1 scan=9677,1,1,154.857910,9677,NDTKEDVFVHQTAIK,15,684.3087,0,1,...,0.0,0.0,33.072314,0.000069,2050.911754,1025.959515,684.308769,513.483396,0.0,723.0
23545,controllerType=0 controllerNumber=1 scan=8704,1,1,149.834488,8704,GGSGGSHGGGSGFGGESGGSYGGGEEASGSGGGYGGGSGK,40,1075.0961,0,1,...,0.0,0.0,0.000000,0.002616,3223.281594,1612.144435,1075.098716,806.575856,0.0,509.0
17656,controllerType=0 controllerNumber=1 scan=7070,1,1,147.861938,7070,SKSEEAHAEDSVMDHHFR,18,692.0070,0,0,...,0.0,0.0,2.067013,0.000544,2765.008345,1383.007811,922.340966,692.007544,0.0,723.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95067,controllerType=0 controllerNumber=1 scan=26855,3,1,1.213224,26855,.(Carbamyl)SPESLLPPMLLQPPQESVEPAGPLDVLGPSLQGRE...,74,1389.3148,0,0,...,0.0,0.0,0.000000,-0.002418,6942.532804,3471.770040,2314.849119,1736.388658,NaN,NaN
106616,controllerType=0 controllerNumber=1 scan=29276,4,1,1.201922,29276,DSAFWLSWGLLYAGFIFIM(Oxidation)ALFLALVIRSTQFIIL...,75,1443.2017,0,0,...,0.0,0.0,0.000000,0.005055,7212.004670,3606.505973,2404.673074,1803.756625,NaN,NaN
95068,controllerType=0 controllerNumber=1 scan=26855,4,0,1.183505,26855,.(Carbamyl)TMPHFARSQTPLPSLWYVSDSSRDVVPSLGVPLTC...,76,1389.3148,0,0,...,0.0,0.0,0.000000,-0.002825,6942.530769,3471.769023,2314.848441,1736.388150,NaN,NaN
95069,controllerType=0 controllerNumber=1 scan=26855,5,1,1.178079,26855,TIPM(Oxidation)STKPANELPLTPRETVVPSVDIISTLACIQP...,77,1389.3148,0,0,...,0.0,0.0,0.000000,-0.005642,6942.516683,3471.761980,2314.843745,1736.384628,NaN,NaN


In [173]:
if dataset[i]['type'] == 'crosslink_data':

    features = ['Score','peplen', 'NuXL:isXL', 'NuXL:modds', 'NuXL:pl_modds', 
                    'NuXL:mass_error_p', 'NuXL:tag_XLed', 'NuXL:tag_unshifted' ,
                    'NuXL:tag_shifted', 'missed_cleavages', 'NuXL:ladder_score',
                    'variable_modifications']
elif dataset[i]['type'] == 'top_down_data':
    features = ['Score', 'NumMass', 'MatchingFragments', 'Coverage(%)', 'TagCount', 'ModCount', 'PrecursorQscore']

# filter data and sort according to descending score
original_df = original_df.filter(np.concatenate([features,['ScanNr', 'rank', 'Label', 'PSMId', 'Peptide','class-specific_q-val', 'cum_target_id']]))
original_df.sort_values('Score',ascending=False, inplace=True)


In [174]:
 # initialise the k-fold cross validator
no_split = 5
kf = KFold(n_splits=no_split, shuffle=True, random_state=1)

param_grid = {
        'svc__C': np.power(float(2), [-5,-1,1,5,7,11,15]),
        #'svc__class_weight': [{1:0.3,0:0.7},{1:0.4,0:0.6},{1:0.5,0:0.5}]
    }

pipe = make_pipeline(MinMaxScaler(), SVC(kernel='linear'))
# create the pipeline
gs = GridSearchCV(pipe,
                param_grid=param_grid, 
                n_jobs=-1,
                scoring="accuracy",
                cv=kf, 
                refit=True,
                verbose = 2)
# filter dataframes
filter_col = 'PSMId'
filter_val = 1

train_idx_list_bottom = []
train_idx_list_top = []
class_weights = []
original_df[f'Score_no_svm'] = original_df['Score']
original_df[f"PSMId_no_svm"] = original_df['PSMId']
original_df[f"class-specific_q-val_no_svm"] = original_df['class-specific_q-val']
original_df[f"cum_target_id_no_svm"] = original_df['cum_target_id']
for iteration in range(1,n_iters + 1):
    
   # sort dataframe according to Score column
    original_df.sort_values('Score',ascending=False, inplace=True)
    if dataset[i]['type'] == 'crosslink_data':
        if pep_filter:
            peptides_top_indices = get_nth_top_scan(original_df ,n=0,group_col= "Peptide", score_col="Score").index
            psm_top_indices = get_nth_top_scan(original_df, n=0,group_col= "ScanNr", score_col= "Score").index
            top_indices = get_nth_top_scan(original_df, n= 0, group_col="ScanNr", score_col="Score").index
        else:
            top_indices = pd.Index(get_top_indices(original_df, "ScanNr", "Score"))
    elif dataset[i]['type'] == 'top_down_data':
        top_indices = pd.Index(get_top_indices(original_df, "ScanNr", "Score"))
    # determine minority class 
    classes = []
    for c in np.unique(original_df[dataset[i]['group']]):
        group_indices = original_df.loc[(original_df[dataset[i]['group']] == c),:].index
        filtered_indices = top_indices.intersection(group_indices)
        classes.append(len(original_df.loc[filtered_indices,:]))
    minority_class = min(classes)
    
    if (minority_class > class_threshold):
        print("Truncating to " + str(class_threshold) + "\n") 
        minority_class = class_threshold

    # define training data (top and bottom scores of each class with PSMId = 1)
    train_idx = []
    train_idx_list_bottom_iter = []
    train_idx_list_top_iter = []
    original_df['train_label'] = np.NaN
    original_df.sort_values('Score',ascending=False, inplace=True)
    
    for c in np.unique(original_df[dataset[i]['group']]):
        
        group_indices = original_df.loc[(original_df[dataset[i]['group']] == c),:].index
        filtered_indices = top_indices.intersection(group_indices)
        # define targets as PSMs with top scores
        class_top = original_df.loc[filtered_indices,:].sort_values('Score',ascending=False)[:int(minority_class/2)].index
        # define decoys with different methods
        if decoy == "bottom": # worst scores
            class_bottom = original_df.loc[filtered_indices,:].sort_values('Score',ascending=False)[-int(minority_class/2):].index
        elif decoy == "last": # last PSM of each target
            top_scans = original_df.loc[class_top,'ScanNr']
            last_decoys = get_nth_top_scan(original_df[original_df['ScanNr'].isin(top_scans)],n=-1, group_col="ScanNr", score_col="Score").index
            last_decoys = last_decoys.difference(class_top)
            class_bottom = last_decoys
        elif decoy == "second": # second PSM of each target
            top_scans = original_df.loc[class_top,'ScanNr']
            second_decoys = get_nth_top_scan(original_df[original_df['ScanNr'].isin(top_scans)],n= 1, group_col="ScanNr", score_col="Score").index
            class_bottom = second_decoys
        elif decoy == "bottom+last": # bottoms and last of each target
            class_bottom = original_df.loc[filtered_indices,:].sort_values('Score',ascending=False)[-int(minority_class/2):].index
            top_scans = original_df.loc[class_top,'ScanNr']
            last_decoys = get_nth_top_scan(original_df[original_df['ScanNr'].isin(top_scans)],n=-1, group_col="ScanNr", score_col="Score").index
            last_decoys = last_decoys.difference(class_top)
            class_bottom = np.concatenate([class_bottom, last_decoys])
        elif decoy == "bottom+second": # bottom and second of each target
            class_bottom = original_df.loc[filtered_indices,:].sort_values('Score',ascending=False)[-int(minority_class/2):].index
            top_scans = original_df.loc[class_top,'ScanNr']
            second_decoys = get_nth_top_scan(original_df[original_df['ScanNr'].isin(top_scans)],n=1,group_col="ScanNr", score_col="Score").index
            class_bottom = np.concatenate([class_bottom, second_decoys])
        original_df.loc[class_top,'train_label'] = 1
        original_df.loc[class_bottom,'train_label'] = 0
        
        train_idx = np.concatenate([train_idx, class_top, class_bottom])
        train_idx_list_bottom_iter = np.concatenate([train_idx_list_bottom_iter, class_bottom])
        train_idx_list_top_iter = np.concatenate([train_idx_list_top_iter, class_top])

    train_idx_list_bottom.append(train_idx_list_bottom_iter) 
    train_idx_list_top.append(train_idx_list_top_iter)
    
    # fit SVM
    gs.fit(original_df.loc[train_idx, features], original_df.loc[train_idx, 'train_label'])
    class_weights.append(gs.best_params_)
    
    # compute new score
    original_df['Score'] = gs.decision_function(original_df.loc[:,features])
    original_df[f'Score_{iteration}'] = original_df['Score']

    # rerank PSMs
    if dataset[i]['type'] != 'top_down_data':
        original_df = rerank(original_df, "ScanNr", "Score", "PSMId")
    original_df[f"PSMId_{iteration}"] = original_df['PSMId']

    # filter for rank = 0
    mask = original_df[original_df[filter_col] == filter_val].index
    
    # compute q-values
    original_df.loc[mask,:] = calcQ(original_df.loc[mask,:], classColName=dataset[i]['group'])
    original_df.loc[mask,f"class-specific_q-val_{iteration}"] = original_df.loc[mask,'class-specific_q-val']
    
    # compute target IDs
    original_df.loc[mask,:] = get_target_id(original_df.loc[mask,:], isXLColName=dataset[i]['group'])
    original_df.loc[mask,f"cum_target_id_{iteration}"] = original_df.loc[mask,'cum_target_id']

Truncating to 500

Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


Truncating to 500

Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


Truncating to 500

Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


Truncating to 500

Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


Truncating to 500

Fitting 5 folds for each of 7 candidates, totalling 35 fits


c:\Users\mail\Research-Project\src\q_value_calc_crosslinks.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[labelColName].replace(to_replace=-1, value=0, inplace=True)


In [175]:
display(original_df)

,Score,peplen,NuXL:isXL,NuXL:modds,NuXL:pl_modds,NuXL:mass_error_p,NuXL:tag_XLed,NuXL:tag_unshifted,NuXL:tag_shifted,missed_cleavages,...,class-specific_q-val_3,cum_target_id_3,Score_4,PSMId_4,class-specific_q-val_4,cum_target_id_4,Score_5,PSMId_5,class-specific_q-val_5,cum_target_id_5
3,0.275532,7,0,12.331852,0.000000,0.882354,0,2,0,0,...,0.319058,1401.0,0.207834,1,0.284444,1350.0,0.275532,1,0.280848,1321.0
0,0.047594,7,0,15.059175,0.000000,0.986818,0,2,0,2,...,NaN,NaN,0.020350,2,NaN,NaN,0.047594,2,NaN,NaN
2,-0.340111,17,0,12.645285,0.000000,0.986815,0,2,0,2,...,NaN,NaN,-0.348649,4,NaN,NaN,-0.340111,3,NaN,NaN
1,-0.340111,17,0,12.645285,0.000000,0.986815,0,2,0,2,...,NaN,NaN,-0.348649,3,NaN,NaN,-0.340111,4,NaN,NaN
6,-0.357047,11,0,11.686532,0.000000,0.475577,0,1,0,1,...,0.816631,3692.0,-0.407874,1,0.809288,3639.0,-0.357047,1,0.790894,3668.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144390,-0.170582,21,1,5.893956,2.404170,0.944509,0,1,0,0,...,0.434913,9034.0,-0.224950,1,0.421859,8683.0,-0.170582,1,0.424321,8536.0
144391,-0.376478,13,1,5.641823,2.350726,0.268126,0,1,1,0,...,NaN,NaN,-0.441469,2,NaN,NaN,-0.376478,2,NaN,NaN
144394,-0.157961,12,1,5.763167,1.214057,0.851936,0,1,0,1,...,0.434228,9016.0,-0.222173,1,0.419660,8657.0,-0.157961,1,0.418100,8431.0
144393,-0.472639,23,1,5.763167,1.214057,0.851936,0,1,0,1,...,NaN,NaN,-0.510364,2,NaN,NaN,-0.472639,2,NaN,NaN


In [176]:
# save entire dataframe
directory=f"../data/{dataset[i]['type']}/{dataset[i]['name']}/{class_threshold}/"
if pep_filter and dataset[i]['type'] == 'crosslink_data':
    directory += "pep_unique/"
else:
    directory += "default/"
if decoy != 'bottom':
    directory += f"decoy_{decoy}/"
if not os.path.exists(directory):
    os.makedirs(directory)
output_file = re.sub('.pkl', f"_svm.pkl", input_filename)
path = directory + output_file
original_df.to_pickle(path)